# Analyse et prédiction des rendements de SCPI

Le but ici est de livrer une analyse et une tentative de prédiction des rendements que peut offir les SCPI.
Pour cela, j'ai extrait et utilisé les données du site https://www.centraledesscpi.com/scpi/

Les parts de SCPI représentent un investissement immobilier dont le risque est modéré et dont les revenus sont assurés par le pourcentage de distribution (loyers) et la plus-value que l'on peut faire au moment de la revente des parts (capital).

On va réaliser :
- 1 - la collecte des données
- 2 - preprocessing des données
- 3 - data analyse
- 4 - prédiction du pourcentage de distribution

# 1. Collecte des données

On va d'abord faire du scrapping de données afin de récupérer tous ce qu'il nous intéresse. La liste ci-dessous représente la fin de l'url qui correspond à la SCPI. On extrait la page internet pour chaque SCPI, puis on passe ligne par ligne pour extraire les infos qui nous intéresse.

In [7]:
import requests
from bs4 import BeautifulSoup

# URL de la page web à scraper

debut_url = "https://www.scpi-online.com/scpi/"

liste = [
'scpi-notapierre',
'scpi-corum-origin'
'allianz-home',
'corum-xl',
'remake-live',
'novapierre-1',
'laffitte-pierre',
'activimmo',
'capiforce',
'buroboutic',
'atream-hotels',
'atout-pierre-diversification',
'affinites-pierre',
'allianz-pierre',
'altixia-commerces',
'amundi-delta-capital-sante',
'epargne-pierre',
'pfo2',
'log-in',
'interpierre-france',
'interpierre-europe-centrale',
'immorente',
'aew-patrimoine-sante',
'aew-paris-commerces',
'carac-perspectives-immo',
'genepierre',
'fructiregions-europe',
'atlantique-pierre-1',
'fructipierre',
'france-investipierre',
'scpi-accimmo-pierre',
'scpi-actipierre-europe',
'altixia-cadence-xii',
'scpi-acces-valeur-pierre',
'scpi-metronome',
'aestiam-pierre-rendement',
'aestiam-placement-pierre',
'cap-foncieres-territoires',
'diversification-allemagne',
'atlantique-murs-regions',
'primovie',
'rivoli-avenir-patrimoine',
'selectinvest-1',
'selectipierre-2',
'sofidy-europe-invest',
'sofipierre',
'sofiprime',
'soprorente',
'transitions-europe',
'ufifrance-immobilier',
'unidelta',
'urban-coeur-commerce',
'vendome-regions',
'fonciere-remusat',
'fonciere-des-praticiens',
'primopierre',
'primofamily',
'pierval-sante',
'pierre-plus',
'pierre-expansion-sante',
'pfo',
'pf-grand-paris',
'ficommerce-proximite',
'aestiam-caphebergimmo',
'fair-invest',
'eurovalys',
'pf-hospitalite-europe',
'esg-pierre-capital',
'epsilon-360',
'epargne-pierre-europe',
'epargne-fonciere',
'elysees-pierre',
'patrimmo-croissance',
'patrimmo-commerce',
'opus-real',
'optimale',
'neo',
'novapierre-residentiel',
'novapierre-allemagne-2',
'novapierre-allemagne',
'elialys',
'efimmo-1',
'edissimmo',
'cristal-rente',
'cristal-life',
'credit-mutuel-pierre-1',
'corum-xl',
'corum-origin',
'corum-eurion',
'coeur-de-ville',
'coeur-de-regions',
'notapierre',
'my-share-scpi',
'logipierre-3',
'lf-opportunite-immo',
'coeur-europe',
'lf-grands-palais',
'lf-grand-paris-patrimoine',
'cm-cic-pierre-investissement',
'lf-europimmo',
'lf-avenir-sante',
'le-patrimoine-foncier',
'loustal-des-aveyronnais',
'kyaneos-pierre',
'iroko-zen',
'immorente-2',
'immo-placement',
'scpi-immo-evolutif',
'scpi-gma-essentialis',
'scpi-france-investipierre'
    
]
i = 0
for scpi in liste :
    
    url = debut_url + scpi
    i+=1
    # Envoie une requête GET à l'URL
    response = requests.get(url)

    # Vérifie si la requête s'est bien déroulée
    if response.status_code == 200:
        # Analyse le contenu HTML de la page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Récupère tout le texte de la page
        texte_page = soup.get_text()

        # Enregistre le texte dans un fichier texte
        with open(str(i)+"_"+scpi+".txt", "w", encoding="utf-8") as fichier:
            fichier.write(texte_page)

        print('Données enregistrées avec succès dans '+str(i)+'_'+scpi+'.txt')

    else:
        print("Erreur lors de la requête HTTP:", response.status_code)

Données enregistrées avec succès dans 1_allianz-home.txt
Données enregistrées avec succès dans 2_corum-xl.txt
Données enregistrées avec succès dans 3_remake-live.txt
Données enregistrées avec succès dans 4_novapierre-1.txt
Données enregistrées avec succès dans 5_laffitte-pierre.txt
Données enregistrées avec succès dans 6_activimmo.txt
Données enregistrées avec succès dans 7_capiforce.txt
Données enregistrées avec succès dans 8_buroboutic.txt
Données enregistrées avec succès dans 9_atream-hotels.txt
Données enregistrées avec succès dans 10_atout-pierre-diversification.txt
Données enregistrées avec succès dans 11_affinites-pierre.txt
Données enregistrées avec succès dans 12_allianz-pierre.txt
Données enregistrées avec succès dans 13_altixia-commerces.txt
Données enregistrées avec succès dans 14_amundi-delta-capital-sante.txt
Données enregistrées avec succès dans 15_epargne-pierre.txt
Données enregistrées avec succès dans 16_pfo2.txt
Données enregistrées avec succès dans 17_log-in.txt
Don

In [5]:
import pandas as pd

from bs4 import BeautifulSoup

# URL de la page web à scraper
debut_url = "https://www.scpi-online.com/scpi/"

i = 0

scpi_liste = []
type_SCPI = []
categorie = []
capital = []
creation = []
capitalisation = []
nb_associes = []
taux_occupation_financier = []
nb_immeubles = []
RAN = []
pct_charge = []

pct_distrib_2022 = []
variation_prix_2022 = []
pct_distrib_2021 = []
variation_prix_2021 = []
pct_distrib_2020 = []
variation_prix_2020 = []
pct_distrib_2019 = []
variation_prix_2019 = []
pct_distrib_2018 = []
variation_prix_2018 = []


for scpi in liste :

    url = debut_url + scpi
    i+=1
    # Envoie une requête GET à l'URL
    response = requests.get(url)

    # Vérifie si la requête s'est bien déroulée
    if response.status_code == 200:
        # Analyse le contenu HTML de la page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Récupère tout le texte de la page
        texte = soup.get_text()

        # traitement du texte     
        # Divisez le texte en lignes
        lignes = texte.strip().split('\n')

        # Initialisez des variables pour stocker les données
        annee = None
        pourcentage_distribution = None
        variation_prix = None

        type_SCPI_var = None
        categorie_var = None       
        capital_var = None
        creation_var = None
        capitalisation_var = None
        nb_associes_var = None
        taux_occupation_financier_var = None
        nb_immeubles_var = None
        RAN_var = None
        pct_charge_var = None
        
        pct_distrib_2022_var=None
        variation_prix_2022_var=None
        pct_distrib_2021_var=None
        variation_prix_2021_var=None
        pct_distrib_2020_var=None
        variation_prix_2020_var=None
        pct_distrib_2019_var=None
        variation_prix_2019_var=None
        pct_distrib_2018_var=None
        variation_prix_2018_var=None

        # Initialisation des flags

        type_SCPI_flag = 0
        categorie_flag = 0       
        capital_flag = 0
        creation_flag = 0
        capitalisation_flag = 0
        nb_associes_flag = 0
        taux_occupation_financier_flag = 0
        nb_immeubles_flag = 0
        RAN_flag = 0
        pct_charge_flag = 0
        
        tableau_passage = False
        #print(texte)
        # Parcourez les lignes pour extraire les données
        for ligne in lignes:
            #print(ligne)


            if type_SCPI_flag == 1:
                type_SCPI_flag = 0
                type_SCPI_var = ligne
                #print("Type",ligne)
            if ligne == 'Type' or ligne == 'Type ':
                type_SCPI_flag = 1
                #print("Type flag",ligne)
                
            if categorie_flag == 1:
                categorie_flag = 0
                categorie_var = ligne
                #print("Catégorie",ligne)
            if (ligne == 'Catégorie' or ligne == 'Catégorie ') and not ':' in ligne :
                categorie_flag = 1
                
                #print("Catégorie flag",ligne)
            if capital_flag == 1:
                capital_flag = 0
                capital_var = ligne
                #print("Capital",ligne)
            if ligne == 'Capital':
                capital_flag = 1

            if creation_flag == 1:
                creation_flag = 0
                creation_var = ligne
                #print("Création",ligne)
            if (ligne == 'Création' or ligne == 'Création ' )and not ':' in ligne :
                creation_flag = 1

            if capitalisation_flag == 1:
                capitalisation_flag = 0
                capitalisation_var = ligne
                #print("Capital",ligne)
            if 'Capitalisation' in ligne and not ':' in ligne  :
                capitalisation_flag = 1

            if nb_associes_flag == 1:
                nb_associes_flag = 0
                nb_associes_var = ligne
                #print("Nombre d'associés",ligne)
            if "Nombre d'associés" in ligne and not ':' in ligne  :
                nb_associes_flag = 1
                
            if taux_occupation_financier_flag == 1:
                taux_occupation_financier_flag = 0
                taux_occupation_financier_var = ligne
                #print("Taux d'occupation financier",ligne)
            if "Taux d'occupation financier" in ligne and not ':' in ligne  :
                taux_occupation_financier_flag = 1

            if nb_immeubles_flag == 1:
                nb_immeubles_flag = 0
                nb_immeubles_var = ligne
                #print("Nombre d'immeubles",ligne)
            if "Nombre d'immeubles" in ligne and not "/ nombre d'immeubles" in ligne  :
                nb_immeubles_flag = 1                         

            if RAN_flag == 1:
                RAN_flag = 0
                RAN_var = ligne
                #print("Nombre d'immeubles",ligne)
            if "RAN en % du dividende" in ligne and not ':' in ligne  :
                RAN_flag = 1 
                
            if pct_charge_flag == 1:
                pct_charge_flag = 0
                pct_charge_var = ligne
                #print("Nombre d'immeubles",ligne)
            if "Pourcentage de charges" in ligne and not ':' in ligne  :
                pct_charge_flag = 1 
                

            if 'Taux de distribution et variation du prix' in ligne:
                tableau_passage = True
                #print('tableau_passage')
            if tableau_passage == True:

                # Vérifier si la ligne contient une année
                if ligne.strip().isdigit():
                    pourcentage_distribution  = None
                    variation_prix  = None
                    annee = int(ligne.strip())
                # Vérifier si la ligne contient un pourcentage de distribution
                elif ligne.strip().endswith('%'):
                    #pourcentage_distribution = ligne.strip() if pourcentage_distribution is None else None
                    #variation_prix = ligne.strip() if variation_prix == None and pourcentage_distribution is not None else None
                    if variation_prix == None and pourcentage_distribution is not None:
                        variation_prix = ligne.strip()
                    if pourcentage_distribution is None:
                        pourcentage_distribution = ligne.strip()

                    
                if annee is not None and pourcentage_distribution is not None and variation_prix is not None:

                    if annee == 2022:
                        pct_distrib_2022_var = pourcentage_distribution
                        variation_prix_2022_var = variation_prix

                    elif annee == 2021:
                        pct_distrib_2021_var = pourcentage_distribution
                        variation_prix_2021_var = variation_prix

                    elif annee == 2020:
                        pct_distrib_2020_var = pourcentage_distribution
                        variation_prix_2020_var = variation_prix

                    elif annee == 2019:
                        pct_distrib_2019_var = pourcentage_distribution
                        variation_prix_2019_var = variation_prix

                    elif annee == 2018:
                        pct_distrib_2018_var = pourcentage_distribution
                        variation_prix_2018_var = variation_prix

        annee = None
        pourcentage_distribution = None
        variation_prix  = None
        
        #on alimente les listes une fois le parcours du texte

        scpi_liste.append(scpi)
        type_SCPI.append(type_SCPI_var)
        categorie.append(categorie_var)

        capital.append(capital_var)
        creation.append(creation_var)
        capitalisation.append(capitalisation_var)
        nb_associes.append(nb_associes_var)
        taux_occupation_financier.append(taux_occupation_financier_var)
        nb_immeubles.append(nb_immeubles_var)
        RAN.append(RAN_var)
        pct_charge.append(pct_charge_var)
        print(scpi)
        
        pct_distrib_2022.append(pct_distrib_2022_var)
        variation_prix_2022.append(variation_prix_2022_var)
        pct_distrib_2021.append(pct_distrib_2021_var)
        variation_prix_2021.append(variation_prix_2021_var)
        pct_distrib_2020.append(pct_distrib_2020_var)
        variation_prix_2020.append(variation_prix_2020_var)
        pct_distrib_2019.append(pct_distrib_2019_var)
        variation_prix_2019.append(variation_prix_2019_var)
        pct_distrib_2018.append(pct_distrib_2018_var)
        variation_prix_2018.append(variation_prix_2018_var)        
        

    else:
        print("Erreur lors de la requête HTTP:", response.status_code)
        
#on créé le dataframe

#print(type_SCPI)
#print(categorie)
#print(capital)
#print(creation)
#print(capitalisation)
#print(nb_associes)
#print(taux_occupation_financier)
#print(nb_immeubles)

df = pd.DataFrame({
'scpi_liste':scpi_liste,
'type_SCPI':type_SCPI,
'categorie':categorie,
'capital':capital,
'creation':creation,
'capitalisation':capitalisation,
'nb_associes':nb_associes,
'taux_occupation_financier':taux_occupation_financier,
'nb_immeubles':nb_immeubles,
'RAN':RAN,
'pct_charge':pct_charge,
'variation_prix_2018':variation_prix_2018,
'variation_prix_2019':variation_prix_2019,
'variation_prix_2020':variation_prix_2020,
'variation_prix_2021':variation_prix_2021,
'variation_prix_2022':variation_prix_2022,
'pct_distrib_2018':pct_distrib_2018,
'pct_distrib_2019':pct_distrib_2019,
'pct_distrib_2020':pct_distrib_2020,
'pct_distrib_2021':pct_distrib_2021,
'pct_distrib_2022':pct_distrib_2022
})    

print(df)        

allianz-home
corum-xl
remake-live
novapierre-1
laffitte-pierre
activimmo
capiforce
buroboutic
atream-hotels
atout-pierre-diversification
affinites-pierre
allianz-pierre
altixia-commerces
amundi-delta-capital-sante
epargne-pierre
pfo2
log-in
interpierre-france
interpierre-europe-centrale
immorente
aew-patrimoine-sante
aew-paris-commerces
carac-perspectives-immo
genepierre
fructiregions-europe
atlantique-pierre-1
fructipierre
france-investipierre
scpi-accimmo-pierre
scpi-actipierre-europe
altixia-cadence-xii
scpi-acces-valeur-pierre
scpi-metronome
aestiam-pierre-rendement
aestiam-placement-pierre
cap-foncieres-territoires
diversification-allemagne
atlantique-murs-regions
primovie
rivoli-avenir-patrimoine
selectinvest-1
selectipierre-2
sofidy-europe-invest
sofipierre
sofiprime
soprorente
transitions-europe
ufifrance-immobilier
unidelta
urban-coeur-commerce
vendome-regions
fonciere-remusat
fonciere-des-praticiens
primopierre
primofamily
pierval-sante
pierre-plus
pierre-expansion-sante
pfo


In [6]:
df.to_excel("df_brut.xlsx")